# La Diabetes

## Lecture des données

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
diab = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
diab.head().T

## Interprétation des paramètres

# Relation entre grossesse et diabetes

In [ ]:
diab.count()    

In [ ]:
preg = (diab.Pregnancies != 0.000)
preg.value_counts()

In [ ]:
diab[preg].head()

On peut compter les femmes enceintes avec diabetes ou non:

In [ ]:
diab[preg].Outcome.value_counts()

## Quelle est la probabilité de qu'une femme enceinte soit atteinte de diabetes?

On défine les booléens *pos, neg, preg_pos* et *preg_neg*.

In [ ]:
pos = (diab.Outcome == 1)
neg = (diab.Outcome == 0)
preg_pos = preg & pos
preg_neg = preg & neg

Calculer las probabilités:

In [ ]:
prob_pos = diab[preg_pos].Pregnancies.count()/diab[preg].Pregnancies.count()
prob_neg = diab[preg_neg].Pregnancies.count()/diab[preg].Pregnancies.count()
print('Probabilité de positive:',prob_pos,'->',100*prob_pos,'%')
print('Probabilité de negative:',prob_neg,'->',100*prob_neg,'%')

## Création des jeux d'apprentissage et de test

Créer les jeux d'apprentissage et de test

In [ ]:
X = diab.drop(['Outcome'], axis=1)
y = diab.Outcome

On sépare le dataset en deux parties :
- un ensemble d'apprentissage (80% des données), qui va permettre d'entraîner le modèle
- un ensemble de test (20% des données), qui va permettre d'estimer la pertinence de la prédiction

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

Appliquer une régression logistique pour classifier sur l'ensemble de test

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

## Mesures de performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

Afficher la matrice de confusion :

In [ ]:
print(confusion_matrix(y_test,y_lr))

La pertinence:

In [ ]:
print(accuracy_score(y_test,y_lr))

Néanmoins cette mesure peut être faussée dans certains cas, en particulier si le nombre de 0 et de 1 est déséquilibré.

On a les estimateurs *precision, recall* et *f1-score*:

In [ ]:
print(classification_report(y_test, y_lr))

Un tableau de couples de probabilités : 

In [ ]:
probas = lr.predict_proba(X_test)

In [ ]:
print(probas)

On met les probabilités de prédiction de la valeur 1 dans un dataframe, avec les valeurs effectives, pour faciliter la visualisation :

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

On affiche la distribution des probabilités de prédiction de 1, et celle des non probabilités de prédiction de 0 :

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

On utilise ces distributions pour construire la *courbe ROC*.

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

In [ ]:
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')    
plt.plot([0,0,1],[0,1,1],'g:') 
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

## Ajustement des hyperparamètres (Random Forests)

On teste les forêts aléatoires :

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
rf1 = ensemble.RandomForestClassifier(n_estimators=10, min_samples_leaf=10, max_features=3)
rf1.fit(X_train, y_train)
y_rf1 = rf.predict(X_test)
print(classification_report(y_test, y_rf1))

La *validation curve*

In [ ]:
from sklearn.model_selection import validation_curve
params = np.arange(1, 300,step=30)
train_score, val_score = validation_curve(rf, X, y, 'n_estimators', params, cv=7)
plt.figure(figsize=(12,12))
plt.plot(params, np.median(train_score, 1), color='blue', label='training score')
plt.plot(params, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('n_estimators')
plt.ylabel('score');

Les paramètres qui donne la meilleure pertinence:

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

On lance l'entrainement :

In [ ]:
rf_gs.fit(X_train, y_train)

On peut voir les paramètres sélectionnés et le score :

In [ ]:
print(rf_gs.best_params_)

On sélectionne le meilleur estimateur :

In [ ]:
rf2 = rf_gs.best_estimator_

In [ ]:
y_rf2 = rf2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf2))

On a amélioré la performance du modèle

### Importance des caractéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caracteristiques')